### Preparation
Change the `res_path` variable to the path of simulator results. 
Change the `acc_path` variable to the path of algorithm evaluation results

In [4]:
res_path = '../../../simulator/example_sim_results/'
acc_path = '../../../algorithm/example_traces/'

### Table IV: Influence of INT8 Quantization on Accuracy and Sparsity

In [5]:
import pandas as pd
import numpy as np

model_display = {'llava_vid': 'Llava-Vid', 'llava_onevision': 'Llava-OV', 'minicpm_v': 'MiniCPM'}
dataset_display = {'videomme': 'VMME', 'mlvu': 'MLVU', 'mvbench': 'MVB'}

df_acc = pd.read_csv(acc_path + '/accuracy.csv')
df_acc = df_acc[df_acc['Models'].isin(model_display.keys()) & 
                 df_acc['Dataset'].isin(dataset_display.keys())]

df_focus_fp16 = pd.read_csv(res_path + '/main_focus.csv')
df_focus_fp16 = df_focus_fp16[df_focus_fp16['model'].isin(model_display.keys()) & 
                               df_focus_fp16['dataset'].isin(dataset_display.keys())]
df_focus_fp16['sparsity'] = df_focus_fp16.apply(
    lambda row: 0.0 if row['dense_ops'] == 0 else (1 - row['num_ops'] / row['dense_ops']) * 100,
    axis=1
)

df_int8 = pd.read_csv(res_path + '/int8_focus.csv')
df_int8 = df_int8[df_int8['model'].isin(model_display.keys()) & 
                   df_int8['dataset'].isin(dataset_display.keys())]
df_int8['sparsity'] = df_int8.apply(
    lambda row: 0.0 if row['dense_ops'] == 0 else (1 - row['num_ops'] / row['dense_ops']) * 100,
    axis=1
)

result = []

for _, acc_row in df_acc.iterrows():
    model_key = acc_row['Models']
    dataset_key = acc_row['Dataset']
    dense_fp16_acc = acc_row['Dense']
    ours_fp16_acc = acc_row['Focus']
    int8_dense_acc = acc_row['INT8_Dense']
    int8_focus_acc = acc_row['INT8_Focus']
    
    # Acc Degrade (FP16 - INT8, positive = degradation)
    dense_degrade = dense_fp16_acc - int8_dense_acc
    ours_degrade = ours_fp16_acc - int8_focus_acc
    
    # FP16 Focus sparsity
    fp16_match = df_focus_fp16[(df_focus_fp16['model'] == model_key) & 
                                (df_focus_fp16['dataset'] == dataset_key)]
    fp16_sparsity = fp16_match.iloc[0]['sparsity'] if not fp16_match.empty else np.nan
    
    # INT8 Focus sparsity
    int8_match = df_int8[(df_int8['model'] == model_key) & 
                          (df_int8['dataset'] == dataset_key)]
    int8_sparsity = int8_match.iloc[0]['sparsity'] if not int8_match.empty else np.nan

    # Sparsity Degrade (FP16 - INT8)
    sparsity_degrade = fp16_sparsity - int8_sparsity if not (np.isnan(fp16_sparsity) or np.isnan(int8_sparsity)) else np.nan

    row = {
        'Models': model_display[model_key],
        'Datasets': dataset_display[dataset_key],
        'Dense_Acc': f'{int8_dense_acc:.2f}',
        'Dense_Degrade': f'{dense_degrade:.2f}',
        'Ours_Acc': f'{int8_focus_acc:.2f}',
        'Ours_Degrade': f'{ours_degrade:.2f}',
        'Ours_Sparsity': f'{int8_sparsity:.2f}' if not np.isnan(int8_sparsity) else 'N/A',
        'Ours_Sparsity_Degrade': f'{sparsity_degrade:.2f}' if not np.isnan(sparsity_degrade) else 'N/A'
    }
    
    result.append(row)

df_result = pd.DataFrame(result)

print("="*120)
print("TABLE IV: Influence of INT8 Quantization on Accuracy and Sparsity")
print("="*120)
print(df_result.to_string(index=False))
print("="*120)

TABLE IV: Influence of INT8 Quantization on Accuracy and Sparsity
   Models Datasets Dense_Acc Dense_Degrade Ours_Acc Ours_Degrade Ours_Sparsity Ours_Sparsity_Degrade
Llava-Vid     VMME     64.22         -0.07    62.33         0.41         82.48                  0.34
Llava-Vid     MLVU     68.21         -0.47    64.94         1.05         78.10                  0.17
Llava-Vid      MVB     59.75          0.58    57.95         0.25         78.04                  0.40
 Llava-OV     VMME     58.70         -0.29    57.44         1.26         81.46                  0.03
 Llava-OV     MLVU     63.38         -0.06    62.41         0.11         78.35                 -0.01
 Llava-OV      MVB     58.55         -0.17    56.90        -0.12         85.35                  0.14
  MiniCPM     VMME     58.63          0.18    57.96         0.34         82.84                  0.03
  MiniCPM     MLVU     55.93         -0.04    53.22         0.37         77.99                  0.02
  MiniCPM      MVB     55